In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt

In [2]:
data_path = '/content/gdrive/MyDrive/Pytorch/'

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))
])

cifar10 = datasets.CIFAR10(data_path, train=True, download=True, transform=transform)

cifar10_val = datasets.CIFAR10(data_path, train=False, download=True, transform=transform)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting /content/gdrive/MyDrive/Pytorch/cifar-10-python.tar.gz to /content/gdrive/MyDrive/Pytorch/
Files already downloaded and verified


In [3]:
isinstance(cifar10, torch.utils.data.Dataset)

True

In [4]:
len(cifar10)

50000

In [5]:
img, label = cifar10[0]

In [6]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=True)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                           shuffle=True)

In [7]:
# model
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        # bài toán phân loại 10 lớp nên output ra 10 nodes
        self.fc2 = nn.Linear(32, 10)
        
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        # flatten về dạng vector để cho vào neural network
        out = out.view(-1, 8 * 8 * 8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [8]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader, val_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
            optimizer.zero_grad()     
            loss.backward() 
            optimizer.step()
            loss_train += loss.item()
        correct = 0
        with torch.no_grad():
            for data in val_loader:
                images, labels = data
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                c = (predicted == labels).squeeze()
                correct += c.sum()
        if epoch == 1 or epoch % 1 == 0:
            print('Epoch {}, Training loss {}, Val accuracy {}'.format(
                epoch,
                loss_train / len(train_loader),
                correct / len(cifar10_val)))

In [9]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr=3e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 30,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
    val_loader = val_loader
)

Epoch 1, Training loss 1.829129269818211, Val accuracy 0.4323999881744385
Epoch 2, Training loss 1.4785756197426936, Val accuracy 0.5080999732017517
Epoch 3, Training loss 1.3295024077758155, Val accuracy 0.5360000133514404
Epoch 4, Training loss 1.2238539397106756, Val accuracy 0.5029000043869019
Epoch 5, Training loss 1.1558191264072037, Val accuracy 0.5713000297546387
Epoch 6, Training loss 1.1051295251797533, Val accuracy 0.59170001745224
Epoch 7, Training loss 1.064897638300191, Val accuracy 0.5800999999046326
Epoch 8, Training loss 1.0347344645911165, Val accuracy 0.5957000255584717
Epoch 9, Training loss 1.0045724774870421, Val accuracy 0.6248000264167786
Epoch 10, Training loss 0.9812240294940636, Val accuracy 0.5877000093460083
Epoch 11, Training loss 0.9603100398464886, Val accuracy 0.6319000124931335
Epoch 12, Training loss 0.9398296770765958, Val accuracy 0.6392999887466431
Epoch 13, Training loss 0.9230167802489931, Val accuracy 0.6037999987602234
Epoch 14, Training loss 0